In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import pickle

In [2]:
writer = SummaryWriter()

In [3]:
# !pip install --user numpy==1.19.2

In [4]:
# !pip install jupyter_contrib_nbextensions

In [5]:
data = pd.read_csv('./Data/PURCHASE ORDER DATA EXTRACT 2012-2015_0.csv')

In [6]:
data.shape

(346018, 31)

## Checking the data types

In [7]:
for c in data.columns:
    print(f'{c}:{data[c].dtypes}')

Creation Date:object
Purchase Date:object
Fiscal Year:object
LPA Number:object
Purchase Order Number:object
Requisition Number:object
Acquisition Type:object
Sub-Acquisition Type:object
Acquisition Method:object
Sub-Acquisition Method:object
Department Name:object
Supplier Code:float64
Supplier Name:object
Supplier Qualifications:object
Supplier Zip Code:object
CalCard:object
Item Name:object
Item Description:object
Quantity:float64
Unit Price:object
Total Price:object
Classification Codes:object
Normalized UNSPSC:float64
Commodity Title:object
Class:float64
Class Title:object
Family:float64
Family Title:object
Segment:float64
Segment Title:object
Location:object


In [8]:
# for c in data.columns:
#     print(f'{c}:{data[c].value_counts()}')

In [9]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
# data['Total Price'].fillna('-1', inplace = True)

In [11]:
# for p in data['Total Price'].to_list():
#     # print(p)
#     if ('$' not in p):
#         print(p)

In [12]:
data['Purchase Date'] = pd.to_datetime(data['Purchase Date'], errors='coerce')

In [13]:
data['Creation Date'] = pd.to_datetime(data['Creation Date'], errors='coerce')

In [14]:
data['Total Price'] = data['Total Price'].str.replace('$','', regex=True).astype(float)

In [15]:
data['Unit Price'] = data['Unit Price'].str.replace('$','', regex=True).astype(float)

## Split Data into train test

In [16]:
train_data, test_data = train_test_split(data,test_size=0.01)

print([train_data.shape, test_data.shape])

[(342557, 31), (3461, 31)]


## Initialize Transformer

In [17]:
from rdt import HyperTransformer

ht = HyperTransformer()

ht.detect_initial_config(data=train_data)

print(ht.get_config())

{
    "sdtypes": {
        "Creation Date": "datetime",
        "Purchase Date": "datetime",
        "Fiscal Year": "categorical",
        "LPA Number": "categorical",
        "Purchase Order Number": "categorical",
        "Requisition Number": "categorical",
        "Acquisition Type": "categorical",
        "Sub-Acquisition Type": "categorical",
        "Acquisition Method": "categorical",
        "Sub-Acquisition Method": "categorical",
        "Department Name": "categorical",
        "Supplier Code": "numerical",
        "Supplier Name": "categorical",
        "Supplier Qualifications": "categorical",
        "Supplier Zip Code": "categorical",
        "CalCard": "categorical",
        "Item Name": "categorical",
        "Item Description": "categorical",
        "Quantity": "numerical",
        "Unit Price": "numerical",
        "Total Price": "numerical",
        "Classification Codes": "categorical",
        "Normalized UNSPSC": "numerical",
        "Commodity Title": "categor

In [18]:
from rdt.transformers import *

ht.set_config(config={
  'sdtypes': {
    'Creation Date': 'datetime',
    'Purchase Date': 'datetime'
  },
  'transformers': {
#     'Creation Date': UnixTimestampEncoder(missing_value_replacement="mean"),
#     'Purchase Date': UnixTimestampEncoder(missing_value_replacement="mean")
      'Creation Date': OptimizedTimestampEncoder(missing_value_replacement='mean'),
      'Purchase Date': OptimizedTimestampEncoder(missing_value_replacement='mean')
  }
})

ht.update_transformers_by_sdtype(sdtype='numerical',
  transformer_name='ClusterBasedNormalizer',
  transformer_parameters={'missing_value_generation': 'from_column',
                          'weight_threshold': 0.01,
                         'enforce_min_max_values': True})

ht.get_config()

{
    "sdtypes": {
        "Creation Date": "datetime",
        "Purchase Date": "datetime",
        "Fiscal Year": "categorical",
        "LPA Number": "categorical",
        "Purchase Order Number": "categorical",
        "Requisition Number": "categorical",
        "Acquisition Type": "categorical",
        "Sub-Acquisition Type": "categorical",
        "Acquisition Method": "categorical",
        "Sub-Acquisition Method": "categorical",
        "Department Name": "categorical",
        "Supplier Code": "numerical",
        "Supplier Name": "categorical",
        "Supplier Qualifications": "categorical",
        "Supplier Zip Code": "categorical",
        "CalCard": "categorical",
        "Item Name": "categorical",
        "Item Description": "categorical",
        "Quantity": "numerical",
        "Unit Price": "numerical",
        "Total Price": "numerical",
        "Classification Codes": "categorical",
        "Normalized UNSPSC": "numerical",
        "Commodity Title": "categor

In [19]:
# copy the original data
# train_data_masked = train_data.copy()
# mask it
# train_data_masked[train_data_mask_ind] = np.nan

# fit the transformer
# train_data_ = ht.fit_transform(train_data)

# Mask 10% of the data
# train_data_mask_ind = np.random.rand(*train_data_.shape)< 0.1

# transform the data
# train_data_ = ht.transform(train_data)
# train_data_masked_ = ht.transform(train_data_masked)

In [20]:
# pickle.dump(ht, open('transformer.p', 'wb'))
# pickle.dump(train_data_, open('train_data_.p', 'wb'))

In [21]:
ht = pickle.load(open('transformer.p', 'rb'))
train_data_ = pickle.load(open('train_data_.p', 'rb'))

train_data_mask_ind = np.random.rand(*train_data_.shape)< 0.1

In [22]:
# train_data_

## Algorithm to generate data from Embedding

In [23]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.tensor(data.values).float()
        self.target = torch.tensor(target)#.float()
        
    def __getitem__(self, idx):
        return self.data[idx,:], self.target[idx,:]
    
    def __len__(self):
        return len(self.data)

In [24]:
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import pickle
import os


class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=kwargs["input_shape"], out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=kwargs['hidden'])
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=kwargs['hidden'], out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=kwargs["input_shape"])
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
def train_autoencoder(model, test_data, optimizer_autoencoder, criterion_mse, criterion_ce, device, batch_size=512, epochs = 500):
    train_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    losses = []
    for epoch in range(epochs):
        loss = 0
        for inputs, ind_masked in train_loader:
            # inputs has all the value that is expected to predict
            # target is the masked index
            
            # reshaped data
            targets = inputs.detach().clone().to(device)
#             ind_masked = ind_masked.view(-1, ind_masked.shape[1]).to(device)
            
            inputs[ind_masked==1] = -1 #float('nan')
            
            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer_autoencoder.zero_grad()

            # compute reconstructions
            outputs = model(inputs.to(device))

            # compute training reconstruction loss
            train_loss = criterion_mse(outputs[ind_masked==1], targets[ind_masked==1])**2 + criterion_mse(outputs, targets)
            #+ criterion_ce(outputs[ind_masked==1], labels)
            
#             print(f'inputs shape: {inputs.shape}\noutputs shape: {outputs.shape}\ntargets shape: {targets.shape}')
#             print(criterion_ce(outputs[ind_masked==1].view(labels.shape[0], labels.shape[1]), labels.softmax(dim=1)))
#             print([outputs[ind_masked==1].shape[0], labels.shape[0]])
    
            # compute accumulated gradients
            train_loss.backward()

            # perform parameter update based on current gradients
            optimizer_autoencoder.step()

            # add the mini-batch training loss to epoch loss
            loss += train_loss.item()
            

        # compute the epoch training loss
        losses.append([epoch, loss])
        
        writer.add_scalar('Loss/train', loss, epoch)
        # display the epoch training loss
        print("epoch : {}/{}, loss = {:.3f}".format(epoch + 1, epochs, loss))

In [25]:
train_dataset = MyDataset(train_data_, train_data_mask_ind)

In [26]:
st = time.time()

# Initialize autoencoder
model = AE(input_shape=train_data_.shape[1], hidden = 10).to(device)

# show model in tensorboard
train_iter = iter(train_dataset)
train_d, train_l = next(train_iter)

writer.add_graph(model, train_d.to(device))
writer.close()

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer_autoencoder = optim.Adam(model.parameters(), lr=1e-3)

# criteria for autoencoder
criterion_mse = nn.MSELoss()
criterion_ce = nn.CrossEntropyLoss(reduction='mean')


# train autoencoder
train_autoencoder(model, train_dataset, optimizer_autoencoder, criterion_mse, criterion_ce, device, batch_size=256, epochs=100)

et = time.time()

print(f'Total Execution time: {et-st}s')

pickle.dump(model, open(f'{os.getcwd()}/autoencoder.p', 'wb'))

epoch : 1/100, loss = 4907482575264835.000
epoch : 2/100, loss = 819358873567668.000
epoch : 3/100, loss = 79703044676718.000
epoch : 4/100, loss = 61459071413314.062
epoch : 5/100, loss = 45696122925750.500
epoch : 6/100, loss = 44498639525817.500
epoch : 7/100, loss = 42346570523085.750
epoch : 8/100, loss = 34931538058851.500
epoch : 9/100, loss = 30364459920546.500
epoch : 10/100, loss = 36721981913552.000
epoch : 11/100, loss = 29626241636031.000
epoch : 12/100, loss = 29599957303688.750
epoch : 13/100, loss = 26864256790087.500
epoch : 14/100, loss = 25085997677108.500
epoch : 15/100, loss = 25403576566378.500
epoch : 16/100, loss = 29315502278888.500
epoch : 17/100, loss = 26926931623547.500
epoch : 18/100, loss = 27770546491063.812
epoch : 19/100, loss = 21709379922146.250
epoch : 20/100, loss = 25386929290558.016
epoch : 21/100, loss = 29173642651772.516
epoch : 22/100, loss = 23934399966366.969
epoch : 23/100, loss = 22247602614846.609
epoch : 24/100, loss = 24636669427599.75

## Test Data

In [27]:
ind_mask_test = np.random.rand(*test_data.shape) < 0.1

In [28]:
(ind_mask_test==True).sum()/(test_data.shape[0]*test_data.shape[1])

0.09987790215395513

In [29]:
test_data_masked = test_data.copy()
test_data_masked[ind_mask_test] = np.nan

In [30]:
pd.options.display.max_columns = 100

In [31]:
test_data_masked.head(1)

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,Department Name,Supplier Code,Supplier Name,Supplier Qualifications,Supplier Zip Code,CalCard,Item Name,Item Description,Quantity,Unit Price,Total Price,Classification Codes,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location
111013,2014-06-18,2014-05-16,2013-2014,NaN,13-5314,NaN,NON-IT Services,Services are specifically exempt by statute,NaN,NaN,"Statewide Health Planning & Development, Offic...",1786916.0,Paul Mueller,NaN,NaN,NO,Contract,Medical Training,1.0,7792.0,7792.0,86101605,86101605.0,Medical vocational training services,86101600.0,Scientific vocational training services,86100000.0,Vocational training,86000000.0,Education and Training Services,NaN


In [32]:
with torch.no_grad():
    test_transformed = ht.transform(test_data_masked)
    out = pd.DataFrame(model(torch.tensor(test_transformed.values.astype('float32')).to(device)).to('cpu').numpy(), columns=test_transformed.columns)
    out_predicted = ht.reverse_transform(out)

C:\Users\mnsah\AppData\Roaming\Python\Python38\site-packages\rdt\transformers\categorical.py:153: UserWarning: The data in column 'Fiscal Year' contains new categories that did not appear during 'fit' (nan). Assigning them random values. If you want to model new categories, please fit the data again using 'fit'.
  warnings.warn(
C:\Users\mnsah\AppData\Roaming\Python\Python38\site-packages\rdt\transformers\categorical.py:153: UserWarning: The data in column 'Purchase Order Number' contains new categories that did not appear during 'fit' (nan, PO 221300000612, 76WD2002, +12 more). Assigning them random values. If you want to model new categories, please fit the data again using 'fit'.
  warnings.warn(
C:\Users\mnsah\AppData\Roaming\Python\Python38\site-packages\rdt\transformers\categorical.py:153: UserWarning: The data in column 'Acquisition Type' contains new categories that did not appear during 'fit' (nan). Assigning them random values. If you want to model new categories, please fit 

In [33]:
# from sklearn.metrics import mean_squared_error

# overall_loss = 0
# for i,ind in enumerate(ind_mask_test):
#     if ind.sum()>0:
#         loss = mean_squared_error(test_data_.iloc[i,ind].values, out.iloc[i,ind].values)
#         print(f'{i}: {loss}')
#         overall_loss += loss

# print(f'Overall mse loss: {overall_loss}')

In [34]:
test_data.head(3)

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,Department Name,Supplier Code,Supplier Name,Supplier Qualifications,Supplier Zip Code,CalCard,Item Name,Item Description,Quantity,Unit Price,Total Price,Classification Codes,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location
111013,2014-06-18,2014-05-16,2013-2014,NaN,13-5314,NaN,NON-IT Services,Services are specifically exempt by statute,Services are specifically exempt by statute,NaN,"Statewide Health Planning & Development, Offic...",1786916.0,Paul Mueller,NaN,NaN,NO,Contract,Medical Training,1.0,7792.0,7792.0,86101605,86101605.0,Medical vocational training services,86101600.0,Scientific vocational training services,86100000.0,Vocational training,86000000.0,Education and Training Services,NaN
265353,2015-05-01,2015-04-09,2014-2015,NaN,NMED532237137,NaN,NON-IT Services,Services are specifically exempt by statute,Services are specifically exempt by statute,NaN,"Rehabilitation, Department of",1708000.0,Mount St. Mary's College - Doheny Campus,NaN,90007,NO,vocational rehabiliation,Summer 2015 fees,1.0,10935.0,10935.0,86101803,86101803.0,Vocational rehabilitation services,86101800.0,In service training and manpower development,86100000.0,Vocational training,86000000.0,Education and Training Services,"90007\n(34.027762, -118.28507)"
118804,2014-05-14,2014-05-14,2013-2014,NaN,X2253,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,"Justice, Department of",1745786.0,Neogen Corporation,NaN,NaN,NO,lab supply,methamphetamine Group (RTU) Forensic kit 480 well,18.0,335.0,6030.0,41106103\n51151729,41106103.0,Deoxyribonucleic acid DNA typing kits,41106100.0,Deoxyribonucleic acid DNA analysis kits,41100000.0,Laboratory and scientific equipment,41000000.0,Laboratory and Measuring and Observing and Tes...,NaN


In [35]:
test_data_masked.head(3)

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,Department Name,Supplier Code,Supplier Name,Supplier Qualifications,Supplier Zip Code,CalCard,Item Name,Item Description,Quantity,Unit Price,Total Price,Classification Codes,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location
111013,2014-06-18,2014-05-16,2013-2014,NaN,13-5314,NaN,NON-IT Services,Services are specifically exempt by statute,NaN,NaN,"Statewide Health Planning & Development, Offic...",1786916.0,Paul Mueller,NaN,NaN,NO,Contract,Medical Training,1.0,7792.0,7792.0,86101605,86101605.0,Medical vocational training services,86101600.0,Scientific vocational training services,86100000.0,Vocational training,86000000.0,Education and Training Services,NaN
265353,2015-05-01,NaT,NaN,NaN,NMED532237137,NaN,NON-IT Services,Services are specifically exempt by statute,Services are specifically exempt by statute,NaN,"Rehabilitation, Department of",1708000.0,NaN,NaN,90007,NO,vocational rehabiliation,Summer 2015 fees,1.0,10935.0,10935.0,86101803,86101803.0,Vocational rehabilitation services,86101800.0,In service training and manpower development,86100000.0,Vocational training,86000000.0,Education and Training Services,"90007\n(34.027762, -118.28507)"
118804,NaT,2014-05-14,2013-2014,NaN,X2253,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,"Justice, Department of",NaN,Neogen Corporation,NaN,NaN,NO,NaN,methamphetamine Group (RTU) Forensic kit 480 well,18.0,335.0,6030.0,41106103\n51151729,41106103.0,Deoxyribonucleic acid DNA typing kits,41106100.0,Deoxyribonucleic acid DNA analysis kits,41100000.0,Laboratory and scientific equipment,41000000.0,Laboratory and Measuring and Observing and Tes...,NaN


In [36]:
out_predicted.head(3)

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,Department Name,Supplier Code,Supplier Name,Supplier Qualifications,Supplier Zip Code,CalCard,Item Name,Item Description,Quantity,Unit Price,Total Price,Classification Codes,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location
0,2011-11-03,2012-03-17,2014-2015,7-11-51-02,4500217746,NaN,NON-IT Goods,NaN,State Price Schedule,Surplus,"Water Resources, Department of",25675.642719,"Unident/Unimed, (USA)",CA-SBE DBE,75204,NO,GSO Acct 55337 (Mental Health) Blnkt PO,The Contractor will develop and provide variou...,54.552782,182396.920855,21307.180271,85161502,1.686032e+07,Mailing seals,1.544033e+07,Water treatment and supply equipment,1.379297e+07,Castings and casting assemblies,5.002450e+07,Manufacturing Components and Supplies,"75204\n(32.801403, -96.789068)"
1,2011-10-09,2012-02-21,2014-2015,7-11-51-02,15-10024,NaN,NON-IT Goods,NaN,State Price Schedule,Surplus,"Water Resources, Department of",25690.317118,"Unident/Unimed, (USA)",CA-SBE DBE,75204,NO,LabelWriter Turbo Printer,transmitter,53.827356,182396.920855,21217.496094,44121505,7.079147e+06,Mens slacks or trousers or shorts,1.544033e+07,Packaged combination meals,1.379315e+07,Castings and casting assemblies,5.002451e+07,Manufacturing Components and Supplies,"75204\n(32.801403, -96.789068)"
2,2011-07-27,2011-12-08,2014-2015,1-09-70-02,1PA2L044/45,NaN,NON-IT Goods,NaN,State Price Schedule,Surplus,"Water Resources, Department of",25739.925255,"Bay Medical Co., Inc",CA-SBE DBE,75204,NO,OSP Job # 127554,"Ink Cartridge, HP 88XL, Black, High Yield",52.959586,182396.920855,21132.018493,44101503,7.079147e+06,Dipping sauces or condiments or spreads or mar...,1.544033e+07,Packaged combination meals,1.379294e+07,Castings and casting assemblies,5.002453e+07,Manufacturing Components and Supplies,"75204\n(32.801403, -96.789068)"
